In [1]:
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input,GlobalMaxPool1D
from music21 import *
import os 

Using TensorFlow backend.


In [6]:
filename = '../dataset_text/miditokens.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
miditokens = [tokens.strip().split(' ') for tokens in miditokens]

In [7]:
tokenizer = Tokenizer() # token -> int
tokenizer.fit_on_texts(miditokens)

In [13]:
sample = miditokens[1]
#print(sample)
#print(tokenizer.texts_to_sequences([sample])[0]) # Example of token input -> int output
#print("Sõna sagedus kogu andmestikus", tokenizer.word_counts['end'])
#print("Sõna -> indeks teisendus", tokenizer.word_index['note:c4:v112'])
#print("Indeks -> sõna teisendus", tokenizer.index_word[55])
#print(len(tokenizer.word_index))
#print(tokenizer.index_word)
#print(tokenizer.word_index)

In [15]:
def text2midi(tokens):
    s = stream.Stream()

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))

        if token.startswith("note") and not token.endswith("OFF"):
            noteDuration = 0
            noteName = splitToken[1]
            noteVelocity = int(splitToken[2][1:])

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.endswith("OFF")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = int(splitToken[2][1:])
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s

text2midi(sample).show("midi")